# First trying to do some statistical analysis on the data using modules from Pandas

In [7]:
# Import necessary libraries
import pandas as pd
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm

# Load the dataset
file_path = r"C:\Users\hta031\Github\FEDProtein\results\FEDPROTEIN_ALL_metrics.csv" 
data = pd.read_csv(file_path)

# Create 'diet_phase' column based on the order and row index
data['diet_phase'] = data.apply(lambda x: 'PR' if (x['order'] == 2 and x.name % 2 == 0) or (x['order'] == 1 and x.name % 2 != 0) else 'NR', axis=1)

# Perform ANOVA on 'grain_meal_size' based on 'sex', 'order', and 'diet_phase'
formula = 'grain_meal_size ~ C(sex) + C(order) + C(diet_phase)'
model = ols(formula, data=data).fit()

# Perform ANOVA
anova_results = anova_lm(model, typ=2)

# Print ANOVA results
print(anova_results)


                 sum_sq    df         F    PR(>F)
C(sex)         0.023635   1.0  2.194261  0.154920
C(order)       0.000774   1.0  0.071830  0.791578
C(diet_phase)  0.015905   1.0  1.476622  0.239189
Residual       0.204657  19.0       NaN       NaN


In [ ]:
file_path = r"C:\Users\hta031\Github\FEDProtein\results\FEDPROTEIN_HOURLY_DATA.csv"

In [28]:
import pandas as pd
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm

# Load the dataset
file_path = r"C:\Users\hta031\Github\FEDProtein\results\FEDPROTEIN_HOURLY_DATA.csv"
data = pd.read_csv(file_path)

# Clean the column names to avoid issues with periods or other special characters
data.columns = data.columns.str.replace('.', '_')

# Define diet phases based on the 'order' column
data['diet_phase'] = data.apply(lambda x: 'NR' if x['order'] == 1 else 'PR', axis=1)

# Focus on the key meal components for analysis
core_columns = ['grain_meal_size', 'pr_meal_size', 'nr_meal_size']

# Drop rows with missing data in these core columns
data_cleaned = data.dropna(subset=core_columns)

# Function to determine the significance level
def significance_asterisk(p_value):
    if p_value < 0.001:
        return '***'
    elif p_value < 0.01:
        return '**'
    elif p_value < 0.05:
        return '*'
    else:
        return ''

# Function to run ANOVA on the cleaned dataset
def run_anova_core(data, col):
    # Descriptive statistics for core columns
    desc_stats = data.groupby(['sex', 'order', 'diet_phase'])[col].agg(['mean', 'std', 'count'])
    
    # Perform ANOVA
    formula = f'{col} ~ C(sex) + C(order) + C(diet_phase) + C(sex):C(diet_phase)'
    model = ols(formula, data=data).fit()
    anova_results = anova_lm(model, typ=2)
    
    # Add significance asterisks to the p-values
    anova_results['Significance'] = anova_results['PR(>F)'].apply(significance_asterisk)
    
    return desc_stats, anova_results

# Run ANOVA on core meal components (grain, pr, nr meal sizes)
anova_results_core = {}
for col in core_columns:
    try:
        anova_results_core[col] = run_anova_core(data_cleaned, col)
    except Exception as e:
        print(f"Skipping column {col} due to error: {e}")

# Displaying results
for col, (desc_stats, anova_results) in anova_results_core.items():
    print(f'Descriptive Statistics for {col}:')
    print(desc_stats)
    print(f'ANOVA results for {col} (sex, diet phase, order):')
    print(anova_results)
    print('\n' + '-'*80 + '\n')


Descriptive Statistics for grain_meal_size:
                          mean       std  count
sex order diet_phase                           
F   1.0   NR          2.933424  0.034583      5
    2.0   PR          2.923539  0.157918      6
M   1.0   NR          2.870862  0.090753      6
    2.0   PR          2.862430  0.099904      6
ANOVA results for grain_meal_size (sex, diet phase, order):
                        sum_sq    df         F    PR(>F) Significance
C(sex)                0.021874   1.0  1.884364  0.185826             
C(order)              0.000267   1.0  0.022958  0.881165             
C(diet_phase)         0.000477   1.0  0.041071  0.841556             
C(sex):C(diet_phase)  0.000003   1.0  0.000260  0.987304             
Residual              0.220560  19.0       NaN       NaN             

--------------------------------------------------------------------------------

Descriptive Statistics for pr_meal_size:
                          mean       std  count
sex order diet_p

In [15]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns
import trompy as tp
import csv
from datetime import datetime, timedelta
from matplotlib.lines import Line2D


# the next two lines will expand the printed tables
pd.options.display.max_rows= 40000
pd.options.display.max_columns= 40000


high_protein_color = (0.749, 0.8, 0.8)
low_protein_color = (0.247, 0.961, 0.949)
# the last element of the tuple is the alpha value
high_protein_color_alpha = (0.749, 0.8, 0.8, 0.1)
low_protein_color_alpha = (0.247, 0.961, 0.949, 0.1)





DATA= r"C:\Users\hta031\Github\FEDProtein\results\FEDPROTEIN_HOURLY_DATA.csv"
pd.read_csv(DATA)

,mouse_id,mouse_id.1,sex,order,grain_meal_size,grain_snack_size,grain_number_of_meals,grain_meal_frequency,grain_number_of_snacks,grain_snack_frequency,grain_hourly_meals,grain_hourly_snacks,grain_hourly_mega_meals,pr_meal_size,pr_snack_size,pr_number_of_meals,pr_meal_frequency,pr_number_of_snacks,pr_snack_frequency,pr_hourly_meals,pr_hourly_snacks,pr_hourly_mega_meals,nr_meal_size,nr_snack_size,nr_number_of_meals,nr_meal_frequency,nr_number_of_snacks,nr_snack_frequency,nr_hourly_meals,nr_hourly_snacks,nr_hourly_mega_meals,grain_meals_per_day,pr_meals_per_day,nr_meals_per_day,grain_snacks_per_day,pr_snacks_per_day,nr_snacks_per_day,grain_mega_meals_per_day,pr_mega_meals_per_day,nr_mega_meals_per_day,grain_pellets_per_day,pr_pellets_per_day,nr_pellets_per_day,all_pellets_per_day,all_meals_per_day,all_snacks_per_day,all_mega_meals_per_day
0,FEDXA01,NaN,M,2.0,2.870968,1.0,93,1.333307,75,1.075247,"[0, 0, 1, 9, 3, 3, 0, 0, 0, 1, 1, 0, 0, 4, 2, ...","[0, 0, 1, 3, 0, 0, 0, 0, 0, 0, 0, 2, 0, 2, 4, ...","[0, 0, 1, 1, 0, 1, 1, 2, 0, 1, 1, 2, 2, 0, 1, ...",2.647368,1.0,190,1.127559,257,1.525172,"[0, 0, 3, 1, 0, 2, 0, 1, 0, 0, 0, 0, 1, 0, 4, ...","[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7, 2, ...","[0, 0, 0, 2, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, ...",3.090909,1.0,88,0.527647,104,0.623583,"[0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 1, ...","[0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 3, ...","[0, 0, 0, 2, 1, 2, 1, 0, 1, 0, 2, 1, 1, 1, 1, ...",43;25;23,35;28;31;24;24;31;14,9;18;18;15;13;9;6,30;18;25,32;44;52;62;28;25;13,13;14;14;19;18;12;13,25;22;18,6;7;8;9;19;19;28,27;24;23;22;25;22;25,340;271;224,161;155;176;184;215;231;240,246;234;222;211;241;223;219,340;271;224;161;155;176;184;215;231;240;246;23...,43;25;23;35;28;31;24;24;31;14;9;18;18;15;13;9;6,30;18;25;32;44;52;62;28;25;13;13;14;14;19;18;1...,25;22;18;6;7;8;9;19;19;28;27;24;23;22;25;22;25
1,FEDXA02,NaN,M,2.0,3.019608,1.0,102,1.436715,58,0.816956,"[0, 0, 1, 7, 2, 1, 1, 0, 0, 1, 0, 2, 1, 4, 3, ...","[0, 0, 4, 4, 2, 2, 0, 0, 0, 0, 0, 0, 0, 2, 1, ...","[0, 0, 1, 2, 0, 2, 0, 1, 0, 2, 0, 2, 0, 2, 1, ...",2.795222,1.0,293,1.748326,154,0.918915,"[0, 0, 1, 1, 0, 1, 0, 0, 0, 2, 0, 1, 2, 4, 4, ...","[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 1, ...","[0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, ...",3.034722,1.0,144,0.869295,99,0.597641,"[0, 0, 1, 3, 0, 0, 0, 0, 0, 1, 1, 0, 3, 0, 0, ...","[0, 0, 0, 0, 0, 0, 1, 2, 1, 1, 0, 1, 0, 3, 0, ...","[0, 0, 0, 1, 2, 2, 1, 1, 0, 0, 1, 0, 1, 2, 3, ...",41;39;22,36;39;54;47;45;37;34,13;17;32;27;23;15;12,29;20;9,10;24;32;32;22;16;17,13;16;16;13;16;12;12,25;11;18,8;4;6;9;9;17;16,26;16;12;15;16;20;20,304;214;194,154;153;204;211;203;226;214,225;171;195;172;182;175;181,304;214;194;154;153;204;211;203;226;214;225;17...,41;39;22;36;39;54;47;45;37;34;13;17;32;27;23;1...,29;20;9;10;24;32;32;22;16;17;13;16;16;13;16;12;12,25;11;18;8;4;6;9;9;17;16;26;16;12;15;16;20;20
2,FEDXA03,NaN,M,2.0,2.895652,1.0,115,1.615326,66,0.927057,"[0, 0, 3, 6, 1, 0, 4, 0, 3, 0, 1, 1, 0, 3, 5, ...","[0, 0, 1, 6, 3, 0, 1, 0, 0, 0, 0, 0, 0, 3, 3, ...","[0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, ...",2.824104,1.0,307,1.822704,211,1.252738,"[0, 0, 0, 1, 1, 3, 0, 0, 0, 0, 0, 1, 2, 5, 4, ...","[0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 3, ...","[0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, ...",3.342342,1.0,111,0.667202,74,0.444801,"[0, 0, 0, 2, 1, 2, 1, 0, 0, 0, 0, 1, 0, 6, 0, ...","[0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, ...","[0, 0, 0, 2, 2, 1, 2, 0, 0, 1, 1, 1, 3, 2, 4, ...",51;32;29,38;32;43;47;50;49;46,19;14;17;14;14;19;14,38;13;15,18;42;43;29;25;26;27,7;10;15;12;12;7;11,11;19;17,5;4;3;14;19;20;24,37;39;30;31;30;23;24,244;243;206,147;149;171;247;293;280;318,352;351;279;288;251;222;228,244;243;206;147;149;171;247;293;280;318;352;35...,51;32;29;38;32;43;47;50;49;46;19;14;17;14;14;1...,38;13;15;18;42;43;29;25;26;27;7;10;15;12;12;7;11,11;19;17;5;4;3;14;19;20;24;37;39;30;31;30;23;24
3,FEDXA04,NaN,M,2.0,2.811024,1.0,127,1.792435,96,1.354912,"[0, 0, 2, 6, 6, 3, 0, 0, 2, 1, 4, 2, 2, 3, 5, ...",

In [2]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm

# Load the dataset
file_path = r"C:\Users\hta031\Github\FEDProtein\results\FEDPROTEIN_HOURLY_DATA.csv"
data = pd.read_csv(file_path)

# Clean the column names to avoid issues with periods or other special characters
data.columns = data.columns.str.replace('.', '_')

# Function to clean semicolon-separated columns by calculating the mean of the values over the 7 days
def clean_semicolon_column(column):
    return column.apply(lambda x: np.mean(list(map(float, x.split(';')))) if isinstance(x, str) else x)

# List of semicolon-separated columns to clean (meal size, meal frequency, etc. for PR and NR)
semicolon_columns = [
    'grain_mega_meals_per_day', 'pr_mega_meals_per_day', 'nr_mega_meals_per_day',
    'grain_meal_size', 'pr_meal_size', 'nr_meal_size',
    'grain_number_of_meals', 'pr_number_of_meals', 'nr_number_of_meals',
    'grain_meal_frequency', 'pr_meal_frequency', 'nr_meal_frequency',
    'grain_number_of_snacks', 'pr_number_of_snacks', 'nr_number_of_snacks',
    'grain_snack_frequency', 'pr_snack_frequency', 'nr_snack_frequency'
]

# Apply the cleaning function to each relevant column
for col in semicolon_columns:
    if col in data.columns:
        data[col] = clean_semicolon_column(data[col])

# Define diet phases based on the 'order' column
data['diet_phase'] = data.apply(lambda x: 'NR' if x['order'] == 1 else 'PR', axis=1)

# Now the data is cleaned and we can proceed with the analysis

# Function to determine the significance level
def significance_asterisk(p_value):
    if p_value < 0.001:
        return '***'
    elif p_value < 0.01:
        return '**'
    elif p_value < 0.05:
        return '*'
    else:
        return ''

# Function to run ANOVA on all meal components and provide descriptive statistics and p-values
def run_anova_with_significance(data, col):
    # Descriptive statistics for each combination of sex, order, and diet phase
    desc_stats = data.groupby(['sex', 'order', 'diet_phase'])[col].agg(['mean', 'std', 'count'])
    
    # Perform ANOVA
    formula = f'{col} ~ C(sex) + C(order) + C(diet_phase) + C(sex):C(diet_phase)'
    model = ols(formula, data=data).fit()
    anova_results = anova_lm(model, typ=2)
    
    # Add significance asterisks to the p-values
    anova_results['Significance'] = anova_results['PR(>F)'].apply(significance_asterisk)
    
    return desc_stats, anova_results

# Running ANOVA and calculating descriptive statistics for each meal component
anova_results_full = {}
for col in semicolon_columns:
    try:
        anova_results_full[col] = run_anova_with_significance(data, col)
    except Exception as e:
        print(f"Skipping column {col} due to error: {e}")

# Displaying results
for col, (desc_stats, anova_results) in anova_results_full.items():
    print(f'Descriptive Statistics for {col}:')
    print(desc_stats)
    print(f'ANOVA results for {col} (sex, diet phase, order):')
    print(anova_results)
    print('\n' + '-'*80 + '\n')

# Save the ANOVA results to a file if needed
# Example:
# anova_results_full_df = pd.concat([anova_results for col, (desc_stats, anova_results) in anova_results_full.items()])
# anova_results_full_df.to_csv('anova_results_full.csv')


Descriptive Statistics for grain_mega_meals_per_day:
                           mean       std  count
sex order diet_phase                            
F   1.0   NR          20.133333  3.532390      5
    2.0   PR          18.333333  5.577734      6
M   1.0   NR          21.388889  3.151131      6
    2.0   PR          17.444444  3.649455      6
ANOVA results for grain_mega_meals_per_day (sex, diet phase, order):
                          sum_sq    df         F    PR(>F) Significance
C(sex)                  0.100208   1.0  0.005918  0.939483             
C(order)                8.836364   1.0  0.521875  0.478841             
C(diet_phase)          48.942801   1.0  2.890556  0.105412             
C(sex):C(diet_phase)    6.569489   1.0  0.387993  0.540761             
Residual              321.707407  19.0       NaN       NaN             

--------------------------------------------------------------------------------

Descriptive Statistics for pr_mega_meals_per_day:
                   

In [3]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm

# Load the dataset
file_path = r"C:\Users\hta031\Github\FEDProtein\results\FEDPROTEIN_HOURLY_DATA.csv"
data = pd.read_csv(file_path)

# Clean the column names to avoid issues with periods or other special characters
data.columns = data.columns.str.replace('.', '_')

# Function to clean semicolon-separated columns by calculating the mean of the values over the 7 days
def clean_semicolon_column(column):
    return column.apply(lambda x: np.mean(list(map(float, x.split(';')))) if isinstance(x, str) else x)

# List of semicolon-separated columns to clean
semicolon_columns = [
    'grain_meal_size', 'pr_meal_size', 'nr_meal_size',
    'grain_number_of_meals', 'pr_number_of_meals', 'nr_number_of_meals',
    'grain_meal_frequency', 'pr_meal_frequency', 'nr_meal_frequency',
    'grain_number_of_snacks', 'pr_number_of_snacks', 'nr_number_of_snacks',
    'grain_snack_frequency', 'pr_snack_frequency', 'nr_snack_frequency',
    'grain_mega_meals_per_day', 'pr_mega_meals_per_day', 'nr_mega_meals_per_day'
]

# Apply the cleaning function to each relevant column
for col in semicolon_columns:
    if col in data.columns:
        data[col] = clean_semicolon_column(data[col])

# Define diet phases based on the 'order' column
data['diet_phase'] = data.apply(lambda x: 'NR' if x['order'] == 1 else 'PR', axis=1)

# Between-subject comparison: Sex, Order, and Diet Phase
# We'll use ANOVA to compare different groups (sex, order, diet phase)
def run_anova(data, col):
    formula = f'{col} ~ C(sex) + C(order) + C(diet_phase) + C(sex):C(diet_phase)'
    model = ols(formula, data=data).fit()
    anova_results = anova_lm(model, typ=2)
    return anova_results

# Within-subject comparison: NR vs PR for each order group (Order 1: NR->PR, Order 2: PR->NR)
def within_subject_comparison(data, col):
    data_order_1 = data[data['order'] == 1]
    data_order_2 = data[data['order'] == 2]
    
    # For Order 1 (NR first, then PR)
    paired_diff_order_1 = data_order_1['nr_' + col] - data_order_1['pr_' + col]
    mean_diff_order_1 = paired_diff_order_1.mean()
    
    # For Order 2 (PR first, then NR)
    paired_diff_order_2 = data_order_2['pr_' + col] - data_order_2['nr_' + col]
    mean_diff_order_2 = paired_diff_order_2.mean()
    
    return mean_diff_order_1, mean_diff_order_2

# List of all meal components for which to run comparisons
meal_components = [
    'meal_size', 'number_of_meals', 'meal_frequency', 
    'number_of_snacks', 'snack_frequency', 'mega_meals_per_day'
]

# Running ANOVA for between-subject comparisons
for col in meal_components:
    print(f"Between-Subject ANOVA Results for {col}:")
    anova_results = run_anova(data, col)
    print(anova_results)
    print('-' * 80)

# Running within-subject comparisons (NR vs PR for each order group)
for col in meal_components:
    print(f"Within-Subject Comparison for {col}:")
    mean_diff_order_1, mean_diff_order_2 = within_subject_comparison(data, col)
    print(f"Mean Difference for Order 1 (NR -> PR): {mean_diff_order_1}")
    print(f"Mean Difference for Order 2 (PR -> NR): {mean_diff_order_2}")
    print('-' * 80)


Between-Subject ANOVA Results for meal_size:


PatsyError: Error evaluating factor: NameError: name 'meal_size' is not defined
    meal_size ~ C(sex) + C(order) + C(diet_phase) + C(sex):C(diet_phase)
    ^^^^^^^^^

In [4]:
# Check the column names to ensure we are using the correct ones
print(data.columns)


Index(['mouse_id', 'mouse_id_1', 'sex', 'order', 'grain_meal_size',
       'grain_snack_size', 'grain_number_of_meals', 'grain_meal_frequency',
       'grain_number_of_snacks', 'grain_snack_frequency', 'grain_hourly_meals',
       'grain_hourly_snacks', 'grain_hourly_mega_meals', 'pr_meal_size',
       'pr_snack_size', 'pr_number_of_meals', 'pr_meal_frequency',
       'pr_number_of_snacks', 'pr_snack_frequency', 'pr_hourly_meals',
       'pr_hourly_snacks', 'pr_hourly_mega_meals', 'nr_meal_size',
       'nr_snack_size', 'nr_number_of_meals', 'nr_meal_frequency',
       'nr_number_of_snacks', 'nr_snack_frequency', 'nr_hourly_meals',
       'nr_hourly_snacks', 'nr_hourly_mega_meals', 'grain_meals_per_day',
       'pr_meals_per_day', 'nr_meals_per_day', 'grain_snacks_per_day',
       'pr_snacks_per_day', 'nr_snacks_per_day', 'grain_mega_meals_per_day',
       'pr_mega_meals_per_day', 'nr_mega_meals_per_day',
       'grain_pellets_per_day', 'pr_pellets_per_day', 'nr_pellets_per_day',
     